### Simpson objective builder function

In [1]:
using Symbolics
using LinearAlgebra
using DynamicPolynomials

Auxilary function for square Frobenius norm of the matrix $\left\| M \right\|_F^2$:

In [2]:
function frobenius_norm2(m)
    return tr(m * m')
end

frobenius_norm2 (generic function with 1 method)

Lindblad master equation is given by
\begin{align}\label{eq:L1}
    \frac{d}{dt}{\rho}(t) = \mathcal{L}\rho(t),
\end{align}

where Lindbladian right hand side operator is given as
\begin{align}
    \mathcal{L} [\rho] = - \frac{i}{\hbar}[H, \rho]+\sum_{j=1}^{\ell}\left[ A_j \rho A_j^\dagger - \frac{1}{2}\left\{ A_j^\dagger A_j, \rho \right\} \right],
\end{align}

In [3]:
function lindblad_rhs(rho, H, A)
    """
    Right hand side of the Lindblad master equation
    """
    return -1im * (H * rho - rho * H) + A * rho * A' - (A' * A  * rho + rho * A' * A) / 2
    
end

lindblad_rhs (generic function with 1 method)

Master equation is integrated over the time interval $[t_i, t_{i+1}]$:
\begin{align}
\rho_{(i
+1)} - \rho_{(i)} = \int_{t_i}^{t_{i+1}} \frac{d}{dt}{\rho}(t) dt   
= \int_{t_i}^{t_{i+1}} \!\!\!  \mathcal{L} [\rho(t)]dt  \nonumber
= \mathcal{L} \left[ \int_{t_i}^{t_{i+1}} \!\!\! \rho(t) dt \right].
\end{align}

Consider objective that include integration with double step:
\begin{align}
\sum_{i=1}^N {
{\left\|
\rho_{(i)} - \rho_{(i-2)}
- {\mathcal{L}} \left[\int^{t_i}_{t_{i-2}}\rho(t)dt \right] \right\| }^2_F } \rightarrow \operatorname{min}
\end{align}

Using the Simpson rule we can evaluate integral:
\begin{align}
\int^{t_i}_{t_{i-2}}\rho(t)dt = \frac{1}{3} \Delta t \left( {\rho(t_{i-2}) + 4 \rho(t_{i-1}) + \rho(t_i)} \right) + O(\Delta t^5)
\end{align}

In [4]:
function simpson_obj(ρ::Vector{Matrix{ComplexF64}}, t, H, A)  
    obj = 0
    for i in 3:length(ρ)
        obj += frobenius_norm2(
            ρ[i] - ρ[i-2] - (t[i]-t[i-1])lindblad_rhs((ρ[i-2] + 4ρ[i-1] + ρ[i])/3, H, A)
        )
    end
    obj = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))
    return obj
end

function simpson_obj(ρ::Array{ComplexF64,3}, t, H, A)  
    obj = 0
    for i in 3:length(ρ)
        obj += frobenius_norm2(
            ρ[:, :, i] - ρ[:, :, i-2] - (t[i]-t[i-1])lindblad_rhs((ρ[:, :, i-2] + 4ρ[:, :, i-1] + ρ[:, :, i])/3, H, A)
        )
    end
    obj = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))
    return obj
end

simpson_obj (generic function with 2 methods)

Consider test example

In [5]:
@polyvar x[1:4]
H = [ 1.0 * x[1]              x[3] + im * x[4]
      x[3] - im * x[4]        x[2]             ]

@polyvar a[1:2, 1:2]
@polyvar b[1:2, 1:2]
A = 1.0 * a + im * b

ρ1 = [.5 -im
     +im  .5]
ρ2 = [.6 0
     0  .4]
ρ3 = [.7 0
     0  .3]
ρ4 = [.9 0
     0  .1]
ρ5 = [.9 0
     0  .1]
ρ6 = [.9 0
     0  .1]

ρ = [ρ1, ρ2, ρ3, ρ4, ρ5, ρ6]
t = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]

obj = simpson_obj(ρ, t, H, A)

0.0005555555555555556a₁₋₁⁴ - 0.0022222222222222222a₁₋₁³a₂₋₂ - 0.008a₁₋₁³b₂₋₁ - 0.006666666666666667a₁₋₁³b₁₋₂ + 0.1958888888888889a₁₋₁²a₂₋₁² - 0.24444444444444446a₁₋₁²a₂₋₁a₁₋₂ + 0.008a₁₋₁²a₂₋₁b₁₋₁ - 0.02266666666666667a₁₋₁²a₂₋₁b₂₋₂ + 0.0811111111111111a₁₋₁²a₁₋₂² + 0.006666666666666667a₁₋₁²a₁₋₂b₁₋₁ + 0.008000000000000002a₁₋₁²a₁₋₂b₂₋₂ + 0.0033333333333333335a₁₋₁²a₂₋₂² + 0.02266666666666667a₁₋₁²a₂₋₂b₂₋₁ + 0.018666666666666668a₁₋₁²a₂₋₂b₁₋₂ + 0.0011111111111111111a₁₋₁²b₁₋₁² - 0.0022222222222222222a₁₋₁²b₁₋₁b₂₋₂ + 0.1958888888888889a₁₋₁²b₂₋₁² + 0.24888888888888888a₁₋₁²b₂₋₁b₁₋₂ + 0.08333333333333333a₁₋₁²b₁₋₂² + 0.0033333333333333335a₁₋₁²b₂₋₂² - 0.24888888888888888a₁₋₁a₂₋₁²a₂₋₂ - 0.008a₁₋₁a₂₋₁²b₂₋₁ - 0.006666666666666668a₁₋₁a₂₋₁²b₁₋₂ + 0.2593333333333333a₁₋₁a₂₋₁a₁₋₂a₂₋₂ + 0.02666666666666667a₁₋₁a₂₋₁a₂₋₂b₂₋₂ - 0.4933333333333333a₁₋₁a₂₋₁b₁₋₁b₁₋₂ + 0.2637777777777778a₁₋₁a₂₋₁b₁₋₂b₂₋₂ - 0.07555555555555556a₁₋₁a₁₋₂²a₂₋₂ - 0.021333333333333336a₁₋₁a₁₋₂²b₂₋₁ - 0.017333333333333333a₁₋₁a₁₋₂²b₁₋₂ - 0.026666

In [6]:
using TSSOS

opt,sol,data = tssos_first(obj, variables(obj));

************************TSSOS************************
TSSOS is launching...
Starting to compute the block structure...
------------------------------------------------------
The sizes of PSD blocks:
[22, 19, 4]
[1, 1, 2]
------------------------------------------------------
Obtained the block structure. The maximal size of blocks is 22.
Assembling the SDP...
There are 267 affine constraints.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 267             
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 4               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - trie

In [7]:
opt,sol,data = tssos_higher!(data);

Starting to compute the block structure...
No higher TSSOS hierarchy!
